## 231205.ipynb 작업 내용 요약
- HN__4_8__19__selected.csv: 새로운 raw 데이터 셋 생성함.  
- 231204_v4 내용 반영
    1) dtype = object 제외 (해당사항 없음.)  
    2) 9, 99, 999 인 녀석들(모름/무응답) null 처리  
    3)  mh_PHQ_S 는 null값인데, 각 문항 응답인 BP_PHQ_1 ~ 9의 값이 있는 경우, 더해서 mh_PHQ_S에 넣어주기 -> 유효한 mh_PHQ_S를 최대한 늘리기 위한 작업  
    4) 결측값 처리 (1) - row 기준 결측 비율이 30% 넘는 행 제외   
    5) `depressed` 변수 생성  
	    1) 기준 (1) - `mh_PHQ_S >= 10`  
	    2) 기준 (2) - `BP_PHQ_9`에 1,2,3점으로 대답한 경우 (자살 시도 또는 생각)    
    6) 여기까지 `depressed` 비율 확인  
	    1) `depressed == 0`: 80,909  
	    2) `depressed == 1`: 1,399  
    7) 잠재적으로 우울한 사람 drop  
	    1) `DF2_pr == null & mh_PHQ_S == null`  
	    2) `DF2_pr != 8 or mh_PHQ != NaN`  
    8) 결측값 처리 (2) - column 기준 결측 비율이 10% 넘는 열 제외  
	    1) shape: (79743, 94)  
    9) 결측값 처리 (3) - fillna  
	    1) num_features 결측값 처리 - KNN  
	    2) onehot_features, label_features 결측값 처리 - 최빈값(most frequent values)  
	    3) `BP6_10 == NaN or BP6_31 == NaN` 이면 해당 행 drop  
		    1) 두 변수 모두 1년간 자살 생각 및 자살 시도 여부를 예 아니오로 응답한 질문,  null 값 채우기에는 위험하기 때문에 일단 null rows drop, ###TODO 현재는 정확률을 일부로 높이기 위해 해당 변수를 X 피쳐에 포함 시킬 것. 추후에는 해당 변수를 아예 제거해야 정확한 실험 가능합니다.   
    10) 피쳐 그룹별 인코딩 작업  
	    1) numeric -> 인코딩 없이 사용, 단 scaling 필요함 (일단 MinMaxScaler 사용)  
	    2) onehot -> onehot 인코딩으로 진행 (categorical)  
	    3) label -> label 인코딩으로 진행 (ordinal)  
		    1. 비해당 값은 수동으로 적절한 라벨로 변경  
    11) Sampling  
	    1) Under sampling - random  
	    2) Over sampling - SMOTE  
    12) Output to csv  

In [116]:
import pandas as pd
import numpy as np

df_new = pd.read_csv('/home/sangjun/work/data/HN__2020__19__selected.csv')

### 데이터셋 설명
HN__4_8__19__selected.csv
- num_features = 연속형 숫자형 변수들
- onehot_features = 예/아니오 거나 응답별로 서로 순서/척도 관계 없이 정말 범주로만 나와있는 변수들 
- label_features = 적음/보통/많음 과 같이 척도 관계가 있는 변수들
- y_related = ["BP_PHQ_1","BP_PHQ_2","BP_PHQ_3","BP_PHQ_4","BP_PHQ_5","BP_PHQ_6","BP_PHQ_7","BP_PHQ_8","BP_PHQ_9","mh_PHQ_S","BP6_10","BP6_31","DF2_pr","DF2_pt"]

- 총 125개의 columns
- 4기 ~ 8기
- 만 19세 이상만 까지 처리해서 저장한 파일: HN__4_8__19__selected.csv

In [117]:
num_features = ["age",
"ainc",
"HE_PLS_15",
"HE_PLS_30",
"HE_mPLS",
"HE_sbp",
"HE_dbp",
"HE_ht",
"HE_wt",
"HE_wc",
"HE_glu",
"HE_HbA1c",
"HE_insulin",
"HE_chol",
"HE_HDL_st2",
"HE_TG",
"HE_LDL_drct",
"HE_ast",
"HE_alt",
"HE_HB",
"HE_HCT",
"HE_BUN",
"HE_crea",
"HE_WBC",
"HE_RBC",
"HE_Bplt",
"HE_Uacid",
"HE_Uph",
"HE_Usg",
"HE_Ucrea"]

In [118]:
onehot_features = ["region",
"sex",
"occp",
"marri_1",
"tins",
"npins",
"D_2_1",
"DI1_pr",
"DI1_pt",
"DI2_pr",
"DI2_pt",
"DI3_pr",
"DI3_pt",
"DJ4_pr",
"DJ4_pt",
"DE1_pr",
"DE1_pt",
"BH9_11",
"BH1",
"BH2_61",
"LQ4_00",
"LQ1_sb",
"LQ2_ab",
"AC1_yr",
"MH1_yr",
"EC_occp",
"EC_stt_1",
"EC_stt_2",
"EC_wht_0",
"BO1_1",
"BO2_1",
"BD1",
"BS8_2",
"BS9_2",
"LW_ms",
"LW_pr",
"LW_oc",
"HE_rPLS",
"HE_Unitr",
"HE_Upro",
"HE_Uglu",
"HE_Uket",
"HE_Ubil",
"HE_Ubld",
"HE_Uro",
"BM1_0",
"BM1_1",
"BM1_2",
"BM1_3",
"BM1_4",
"BM1_5",
"BM1_6",
"BM1_7",
"BM1_8",
"L_BR_TO",
"L_LN_TO",
"L_DN_TO",
"LK_LB_US",
"LK_LB_EF",
"live_t",
"educ",
"BO1",
"HE_obe",]

In [119]:
label_features = ["incm",
"ho_incm",
"incm5",
"ho_incm5",
"edu",
"cfam",
"house",
"D_1_1",
"DI3_2",
"BD1_11",
"BD2_1",
"BA2_12",
"BA2_13",
"BP1",
"BS3_1",
"BE3_31",
"BE5_1",
"OR1",
]

In [120]:
y_related =["BP_PHQ_1",
"BP_PHQ_2",
"BP_PHQ_3",
"BP_PHQ_4",
"BP_PHQ_5",
"BP_PHQ_6",
"BP_PHQ_7",
"BP_PHQ_8",
"BP_PHQ_9",
"mh_PHQ_S",
"BP6_10",
"BP6_31",
"DF2_pr",
"DF2_pt"]

In [121]:
df = df_new
df

,age,ainc,HE_PLS_15,HE_PLS_30,HE_mPLS,HE_sbp,HE_dbp,HE_ht,HE_wt,HE_wc,...,BP_PHQ_5,BP_PHQ_6,BP_PHQ_7,BP_PHQ_8,BP_PHQ_9,mh_PHQ_S,BP6_10,BP6_31,DF2_pr,DF2_pt
0,39.0,300.000000,NaN,32.0,NaN,116.0,79.0,179.7,78.1,85.8,...,0.0,0.0,0.0,0.0,0.0,0.0,8.0,2.0,8.0,8.0
1,39.0,300.000000,NaN,30.0,NaN,96.0,64.0,171.9,53.0,68.0,...,0.0,0.0,0.0,0.0,0.0,0.0,8.0,2.0,8.0,8.0
2,60.0,1000.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,9.0,9.0,9.0,9.0,9.0,NaN,8.0,2.0,9.0,9.0
3,58.0,1000.000000,NaN,34.0,NaN,104.0,69.0,154.9,63.8,82.0,...,0.0,0.0,0.0,0.0,0.0,0.0,8.0,2.0,8.0,8.0
4,56.0,1373.333333,NaN,30.0,NaN,117.0,68.0,169.7,68.2,86.0,...,0.0,0.0,0.0,0.0,0.0,0.0,8.0,2.0,8.0,8.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6128,61.0,308.333333,NaN,35.0,NaN,113.0,66.0,155.2,57.6,83.5,...,0.0,0.0,0.0,0.0,0.0,0.0,8.0,2.0,8.0,8.0
6129,51.0,500.000000,NaN,35.0,NaN,115.0,71.0,144.7,51.8,73.9,...,0.0,0.0,0.0,0.0,0.0,3.0,8.0,2.0,8.0,8.0
6130,24.0,500.000000,NaN,43.0,NaN,102.0,71.0,148.6,57.7,75.5,...,0.0,0.0,0.0,0.0,0.0,3.0,8.0,2.0,8.0,8.0
6131,65.0,196.666667,NaN,NaN,55.0,139.0,70.0,171.5,76.3,92.1,...,0.0,0.0,0.0,0.0,0.0,0.0,8.0,2.0,8.0,8.0


In [122]:
df["mh_PHQ_S"].isnull().sum()

784

In [123]:
## 20231204_v4 반영
## ['LQ4_24', 'N_DT_DS', 'N_DUSUAL', 'N_DAY'] object 타입이라 뺌
## [9, 99, 999, 9999] 무응답 널처리

####### 얘네 빼도 되는지 확인 - object 타입인 녀석들 ####
# df = df.drop(columns=['LQ4_24', 'N_DT_DS', 'N_DUSUAL', 'N_DAY'], inplace=False)

####### 9, 99, 999 인 녀석들(모름/무응답) null 처리
df = df.replace([9, 99, 999, 9999], np.nan, inplace=False)

In [124]:
cond = df["BP_PHQ_1"].notna() & \
        df["BP_PHQ_2"].notna() & \
        df["BP_PHQ_3"].notna() & \
        df["BP_PHQ_4"].notna() & \
        df["BP_PHQ_5"].notna() & \
        df["BP_PHQ_6"].notna() & \
        df["BP_PHQ_7"].notna() & \
        df["BP_PHQ_8"].notna() & \
        df["BP_PHQ_9"].notna()

df[df["mh_PHQ_S"].isnull() & cond][["BP_PHQ_1", "BP_PHQ_2", "BP_PHQ_3", "BP_PHQ_4", "BP_PHQ_5", "BP_PHQ_6", "BP_PHQ_7", "BP_PHQ_8", "BP_PHQ_9"]].value_counts()

BP_PHQ_1  BP_PHQ_2  BP_PHQ_3  BP_PHQ_4  BP_PHQ_5  BP_PHQ_6  BP_PHQ_7  BP_PHQ_8  BP_PHQ_9
0.0       0.0       3.0       3.0       3.0       0.0       0.0       0.0       0.0         6
3.0       0.0       3.0       3.0       0.0       0.0       0.0       0.0       0.0         3
0.0       3.0       3.0       3.0       0.0       0.0       0.0       0.0       0.0         2
1.0       1.0       3.0       3.0       0.0       0.0       1.0       0.0       0.0         2
                    1.0       1.0       1.0       1.0       1.0       1.0       1.0         2
                                                                                           ..
                              3.0       2.0       1.0       0.0       0.0       0.0         1
                                        3.0       0.0       0.0       0.0       0.0         1
                    2.0       1.0       0.0       1.0       3.0       0.0       0.0         1
                                        1.0       0.0       3.0  

In [125]:
# mh_PHQ_S 는 null값인데, 각 문항 응답인 BP_PHQ_1 ~ 9의 값이 있는 경우, 더해서 mh_PHQ_S에 넣어주기
a = df[df["mh_PHQ_S"].isnull() & cond][["BP_PHQ_1", "BP_PHQ_2", "BP_PHQ_3", "BP_PHQ_4", "BP_PHQ_5", "BP_PHQ_6", "BP_PHQ_7", "BP_PHQ_8", "BP_PHQ_9"]]

b = a[a <= 3].sum(axis=1)
b[b >= 1] #125

df['mh_PHQ_S'] = b[b >=1]

In [126]:
df["EC_occp"].isnull().sum()

1224

In [127]:
### 결측값 처리 1
# row 기준 
threshold = 0.3 # threshold for null proportion
null_percent = df.isnull().mean(axis = 1) # 각 행에 대한 결측값 비율 계산
df = df[null_percent < threshold]
df

,age,ainc,HE_PLS_15,HE_PLS_30,HE_mPLS,HE_sbp,HE_dbp,HE_ht,HE_wt,HE_wc,...,BP_PHQ_5,BP_PHQ_6,BP_PHQ_7,BP_PHQ_8,BP_PHQ_9,mh_PHQ_S,BP6_10,BP6_31,DF2_pr,DF2_pt
0,39.0,300.000000,NaN,32.0,NaN,116.0,79.0,179.7,78.1,85.8,...,0.0,0.0,0.0,0.0,0.0,NaN,8.0,2.0,8.0,8.0
1,39.0,300.000000,NaN,30.0,NaN,96.0,64.0,171.9,53.0,68.0,...,0.0,0.0,0.0,0.0,0.0,NaN,8.0,2.0,8.0,8.0
3,58.0,1000.000000,NaN,34.0,NaN,104.0,69.0,154.9,63.8,82.0,...,0.0,0.0,0.0,0.0,0.0,NaN,8.0,2.0,8.0,8.0
4,56.0,1373.333333,NaN,30.0,NaN,117.0,68.0,169.7,68.2,86.0,...,0.0,0.0,0.0,0.0,0.0,NaN,8.0,2.0,8.0,8.0
5,53.0,1373.333333,NaN,34.0,NaN,102.0,70.0,155.4,47.5,70.7,...,0.0,0.0,0.0,0.0,0.0,NaN,8.0,2.0,8.0,8.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6128,61.0,308.333333,NaN,35.0,NaN,113.0,66.0,155.2,57.6,83.5,...,0.0,0.0,0.0,0.0,0.0,NaN,8.0,2.0,8.0,8.0
6129,51.0,500.000000,NaN,35.0,NaN,115.0,71.0,144.7,51.8,73.9,...,0.0,0.0,0.0,0.0,0.0,NaN,8.0,2.0,8.0,8.0
6130,24.0,500.000000,NaN,43.0,NaN,102.0,71.0,148.6,57.7,75.5,...,0.0,0.0,0.0,0.0,0.0,NaN,8.0,2.0,8.0,8.0
6131,65.0,196.666667,NaN,NaN,55.0,139.0,70.0,171.5,76.3,92.1,...,0.0,0.0,0.0,0.0,0.0,NaN,8.0,2.0,8.0,8.0


In [128]:
df["EC_occp"].isnull().sum()

512

In [129]:
# 생성변수: `depressed` Define depressed (dependent variable)
cond1 = df['mh_PHQ_S'] >= 10 # mh_PHQ_S - total score 
cond2 = df['BP_PHQ_9'].isin([1, 2, 3]) # person who chose 1, 2, 3 in 9th question 

df[cond1 | cond2].drop_duplicates() # person with cond1 or cond2

,age,ainc,HE_PLS_15,HE_PLS_30,HE_mPLS,HE_sbp,HE_dbp,HE_ht,HE_wt,HE_wc,...,BP_PHQ_5,BP_PHQ_6,BP_PHQ_7,BP_PHQ_8,BP_PHQ_9,mh_PHQ_S,BP6_10,BP6_31,DF2_pr,DF2_pt
63,52.0,820.000000,NaN,NaN,49.0,133.0,92.0,159.7,62.4,86.4,...,3.0,3.0,3.0,3.0,3.0,NaN,8.0,2.0,8.0,8.0
90,53.0,698.333333,NaN,30.0,NaN,93.0,63.0,150.9,47.9,70.7,...,1.0,1.0,1.0,0.0,1.0,NaN,8.0,2.0,1.0,0.0
103,27.0,250.000000,NaN,33.0,NaN,123.0,82.0,167.2,51.3,70.6,...,0.0,2.0,0.0,1.0,1.0,NaN,8.0,2.0,8.0,8.0
130,43.0,170.000000,NaN,37.0,NaN,122.0,97.0,173.2,98.9,110.9,...,1.0,2.0,0.0,0.0,1.0,NaN,8.0,2.0,8.0,8.0
192,27.0,530.000000,NaN,46.0,NaN,114.0,83.0,175.2,108.9,117.2,...,3.0,3.0,0.0,1.0,1.0,NaN,8.0,2.0,8.0,8.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5986,55.0,84.000000,NaN,49.0,NaN,131.0,89.0,167.3,83.0,111.1,...,0.0,3.0,0.0,3.0,3.0,NaN,8.0,1.0,8.0,8.0
5994,28.0,385.000000,NaN,40.0,NaN,119.0,85.0,143.5,59.2,84.0,...,3.0,1.0,0.0,0.0,1.0,NaN,8.0,1.0,8.0,8.0
6000,29.0,180.000000,NaN,47.0,NaN,114.0,79.0,162.2,67.1,87.9,...,3.0,1.0,1.0,3.0,1.0,NaN,8.0,2.0,0.0,0.0
6043,71.0,200.000000,NaN,36.0,NaN,132.0,76.0,161.4,57.6,79.8,...,1.0,1.0,1.0,1.0,2.0,NaN,8.0,2.0,8.0,8.0


In [130]:
# Create a new variable 'depressed' 
df.loc[cond1 | cond2, 'depressed'] = 1 # 두 조건 중 하나 이상을 만족하는 행에 'depressed' 변수에 1 값 
df.loc[~(cond1 | cond2), 'depressed'] = 0 # 두 조건 중 하나라도 만족하지 않는 행에 0 값

In [131]:
# Check distribution of depressed to see if the data is imbalanced
df.groupby('depressed').count() # groupby를 이용해 타겟 y 라벨의 클래스 별 개수를 확인

,age,ainc,HE_PLS_15,HE_PLS_30,HE_mPLS,HE_sbp,HE_dbp,HE_ht,HE_wt,HE_wc,...,BP_PHQ_5,BP_PHQ_6,BP_PHQ_7,BP_PHQ_8,BP_PHQ_9,mh_PHQ_S,BP6_10,BP6_31,DF2_pr,DF2_pt
depressed,,,,,,,,,,,,,,,,,,,,,
0.0,5123,5105,0,4619,474,5000,5051,5069,5111,5092,...,5091,5088,5089,5090,5091,59,5123,5123,5120,5120
1.0,292,292,0,262,29,285,288,283,290,288,...,292,290,291,292,292,18,292,292,292,292


In [132]:
# Remove people who might be depressed
# 1) DF2_pr 도 null값이고, mh_PHQ_S 도 null값인 경우 이 사람에 대한 우울증 관련 정보가 없기 때문에 데이터에서 drop 
df.drop(df[df['DF2_pr'].isna() & df['mh_PHQ_S'].isna()].index, inplace=True)

In [133]:
df.shape # Originally from 82308 rows × 125 columns to 82304 rows × 126 columns

(5412, 126)

In [134]:
# 2) DF2_pr == 8 (never been diagnosed by doctor) & mh_PHQ_S == NaN) 인 경우를 제외하려면?
# DF2_pr != 8 or mh_PHQ != NaN (전부 다 ~ 취하기)
df = df[(df['DF2_pr'] != 8 |  ~df['mh_PHQ_S'].isna())]
df.shape

(5226, 126)

In [135]:
df["EC_occp"].isnull().sum()

490

In [136]:
### 결측값 처리 2
# Remove columns if percentage of NaN exceeds 0.1
# threshold = 0.3 # 이거로 하나, 0.1로 하나 큰 차이 x
threshold = 0.1 

null_percent = df.isnull().mean() # 각 열에 대한 결측값 비율 계산
selected_cols = null_percent[null_percent < threshold].index
df = df[selected_cols]
df.shape # (20492, 314)

(5226, 113)

In [137]:
# new_columns_set

columns_list = [
    'age', 'ainc', 'HE_PLS_30', 'HE_sbp', 'HE_dbp', 'HE_ht', 'HE_wt', 'HE_wc', 'HE_glu', 'HE_HbA1c',
    'HE_insulin', 'HE_chol', 'HE_HDL_st2', 'HE_TG', 'HE_ast', 'HE_alt', 'HE_HB', 'HE_HCT', 'HE_BUN',
    'HE_crea', 'HE_WBC', 'HE_RBC', 'HE_Bplt', 'HE_Uacid', 'HE_Uph', 'HE_Usg', 'HE_Ucrea', 'sex',
    'occp', 'marri_1', 'tins', 'npins', 'D_2_1', 'DI1_pr', 'DI1_pt', 'DI2_pr', 'DI2_pt', 'DI3_pr',
    'DI3_pt', 'DJ4_pr', 'DJ4_pt', 'DE1_pr', 'DE1_pt', 'BH9_11', 'BH1', 'BH2_61', 'LQ4_00', 'LQ1_sb',
    'LQ2_ab', 'AC1_yr', 'MH1_yr', 'EC_occp', 'EC_stt_1', 'EC_stt_2', 'EC_wht_0', 'BO1_1', 'BO2_1',
    'BD1', 'BS8_2', 'BS9_2', 'HE_rPLS', 'HE_Unitr', 'HE_Upro', 'HE_Uglu', 'HE_Uket', 'HE_Ubil',
    'HE_Ubld', 'HE_Uro', 'BM1_0', 'BM1_1', 'BM1_2', 'BM1_3', 'BM1_4', 'BM1_5', 'BM1_6', 'BM1_7',
    'BM1_8', 'live_t', 'educ', 'BO1', 'HE_obe', 'incm', 'ho_incm', 'incm5', 'ho_incm5', 'edu',
    'cfam', 'house', 'D_1_1', 'DI3_2', 'BD1_11', 'BD2_1', 'BA2_12', 'BA2_13', 'BP1', 'BS3_1',
    'BE3_31', 'BE5_1', 'OR1', 'BP_PHQ_1', 'BP_PHQ_2', 'BP_PHQ_3', 'BP_PHQ_4', 'BP_PHQ_5', 'BP_PHQ_6',
    'BP_PHQ_7', 'BP_PHQ_8', 'BP_PHQ_9', 'BP6_10', 'BP6_31', 'DF2_pr', 'DF2_pt', 'depressed'
]

In [138]:
# origin_columns_set

origin_columns = ['age', 'ainc', 'HE_sbp', 'HE_dbp', 'HE_ht', 'HE_wt', 'HE_wc', 'HE_glu',
       'HE_chol', 'HE_HDL_st2', 'HE_TG', 'HE_ast', 'HE_alt', 'HE_HB', 'HE_HCT',
       'HE_BUN', 'HE_crea', 'HE_WBC', 'HE_RBC', 'HE_Bplt', 'HE_Uph', 'HE_Usg',
       'sex', 'occp', 'marri_1', 'tins', 'npins', 'D_2_1', 'DI1_pr', 'DI1_pt',
       'DI2_pr', 'DI2_pt', 'DI3_pr', 'DI3_pt', 'DJ4_pr', 'DJ4_pt', 'DE1_pr',
       'DE1_pt', 'BH9_11', 'BH1', 'BH2_61', 'LQ4_00', 'LQ1_sb', 'LQ2_ab',
       'EC_occp', 'EC_stt_1', 'EC_stt_2', 'BO1_1', 'BO2_1', 'BD1', 'BS8_2',
       'BS9_2', 'HE_rPLS', 'HE_Unitr', 'HE_Upro', 'HE_Uglu', 'HE_Uket',
       'HE_Ubil', 'HE_Ubld', 'HE_Uro', 'BM1_1', 'BM1_2', 'BM1_3', 'BM1_4',
       'BM1_5', 'BM1_6', 'BM1_7', 'BM1_8', 'live_t', 'educ', 'BO1', 'HE_obe',
       'incm', 'ho_incm', 'incm5', 'ho_incm5', 'edu', 'cfam', 'house', 'D_1_1',
       'DI3_2', 'BD1_11', 'BD2_1', 'BA2_12', 'BA2_13', 'BP1', 'BS3_1',
       'BE3_31', 'BE5_1', 'BP6_10', 'BP6_31', 'DF2_pr', 'DF2_pt', 'depressed']

In [139]:
# train_columns_set

train_columns = ['age', 'ainc', 'HE_sbp', 'HE_dbp', 'HE_ht', 'HE_wt', 'HE_wc', 'HE_glu',
       'HE_chol', 'HE_HDL_st2', 'HE_TG', 'HE_ast', 'HE_alt', 'HE_HB', 'HE_HCT',
       'HE_BUN', 'HE_crea', 'HE_WBC', 'HE_RBC', 'HE_Bplt', 'HE_Uph', 'HE_Usg',
       'sex', 'occp', 'marri_1', 'tins', 'npins', 'D_2_1', 'DI1_pr', 'DI1_pt',
       'DI2_pr', 'DI2_pt', 'DI3_pr', 'DI3_pt', 'DJ4_pr', 'DJ4_pt', 'DE1_pr',
       'DE1_pt', 'BH9_11', 'BH1', 'BH2_61', 'LQ4_00', 'LQ1_sb', 'LQ2_ab',
       'EC_occp', 'EC_stt_1', 'EC_stt_2', 'BO1_1', 'BO2_1', 'BD1', 'BS8_2',
       'BS9_2', 'HE_rPLS', 'HE_Unitr', 'HE_Upro', 'HE_Uglu', 'HE_Uket',
       'HE_Ubil', 'HE_Ubld', 'HE_Uro', 'BM1_1', 'BM1_2', 'BM1_3', 'BM1_4',
       'BM1_5', 'BM1_6', 'BM1_7', 'BM1_8', 'live_t', 'educ', 'BO1', 'HE_obe',
       'incm', 'ho_incm', 'incm5', 'ho_incm5', 'edu', 'cfam', 'house', 'D_1_1',
       'DI3_2', 'BD1_11', 'BD2_1', 'BA2_12', 'BA2_13', 'BP1', 'BS3_1',
       'BE3_31', 'BE5_1', 'BP6_10', 'BP6_31', 'DF2_pr', 'DF2_pt', 'depressed']

In [140]:
# 1206 train 데이터와 일치

df[['age', 'ainc', 'HE_sbp', 'HE_dbp', 'HE_ht', 'HE_wt', 'HE_wc', 'HE_glu',
       'HE_chol', 'HE_HDL_st2', 'HE_TG', 'HE_ast', 'HE_alt', 'HE_HB', 'HE_HCT',
       'HE_BUN', 'HE_crea', 'HE_WBC', 'HE_RBC', 'HE_Bplt', 'HE_Uph', 'HE_Usg',
       'sex', 'occp', 'marri_1', 'tins', 'npins', 'D_2_1', 'DI1_pr', 'DI1_pt',
       'DI2_pr', 'DI2_pt', 'DI3_pr', 'DI3_pt', 'DJ4_pr', 'DJ4_pt', 'DE1_pr',
       'DE1_pt', 'BH9_11', 'BH1', 'BH2_61', 'LQ4_00', 'LQ1_sb', 'LQ2_ab',
       'EC_occp', 'EC_stt_1', 'EC_stt_2', 'BO1_1', 'BO2_1', 'BD1', 'BS8_2',
       'BS9_2', 'HE_rPLS', 'HE_Unitr', 'HE_Upro', 'HE_Uglu', 'HE_Uket',
       'HE_Ubil', 'HE_Ubld', 'HE_Uro', 'BM1_1', 'BM1_2', 'BM1_3', 'BM1_4',
       'BM1_5', 'BM1_6', 'BM1_7', 'BM1_8', 'live_t', 'educ', 'BO1', 'HE_obe',
       'incm', 'ho_incm', 'incm5', 'ho_incm5', 'edu', 'cfam', 'house', 'D_1_1',
       'DI3_2', 'BD1_11', 'BD2_1', 'BA2_12', 'BA2_13', 'BP1', 'BS3_1',
       'BE3_31', 'BE5_1', 'BP6_10', 'BP6_31', 'DF2_pr', 'DF2_pt', 'depressed']]

,age,ainc,HE_sbp,HE_dbp,HE_ht,HE_wt,HE_wc,HE_glu,HE_chol,HE_HDL_st2,...,BA2_13,BP1,BS3_1,BE3_31,BE5_1,BP6_10,BP6_31,DF2_pr,DF2_pt,depressed
0,39.0,300.000000,116.0,79.0,179.7,78.1,85.8,84.0,150.0,44.0,...,5.0,3.0,3.0,8.0,1.0,8.0,2.0,8.0,8.0,0.0
1,39.0,300.000000,96.0,64.0,171.9,53.0,68.0,89.0,187.0,65.0,...,5.0,3.0,8.0,1.0,1.0,8.0,2.0,8.0,8.0,0.0
3,58.0,1000.000000,104.0,69.0,154.9,63.8,82.0,109.0,208.0,66.0,...,4.0,4.0,8.0,6.0,6.0,8.0,2.0,8.0,8.0,0.0
4,56.0,1373.333333,117.0,68.0,169.7,68.2,86.0,107.0,241.0,57.0,...,4.0,3.0,8.0,6.0,6.0,8.0,2.0,8.0,8.0,0.0
5,53.0,1373.333333,102.0,70.0,155.4,47.5,70.7,90.0,270.0,78.0,...,5.0,3.0,8.0,8.0,1.0,8.0,2.0,8.0,8.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6128,61.0,308.333333,113.0,66.0,155.2,57.6,83.5,98.0,197.0,48.0,...,5.0,3.0,8.0,1.0,1.0,8.0,2.0,8.0,8.0,0.0
6129,51.0,500.000000,115.0,71.0,144.7,51.8,73.9,95.0,192.0,51.0,...,2.0,3.0,8.0,4.0,3.0,8.0,2.0,8.0,8.0,0.0
6130,24.0,500.000000,102.0,71.0,148.6,57.7,75.5,98.0,153.0,71.0,...,3.0,3.0,1.0,6.0,1.0,8.0,2.0,8.0,8.0,0.0
6131,65.0,196.666667,139.0,70.0,171.5,76.3,92.1,119.0,135.0,50.0,...,5.0,3.0,3.0,8.0,6.0,8.0,2.0,8.0,8.0,0.0


In [141]:
df = df[train_columns]

In [142]:
df.columns # 'DF2_pr', 'DF2_pt', 'depressed'는 살아 있어야 함 
#TODO
# DF2_pr 변수 및 DF2_pt 변수 고려해줄 건지... 현 상태에서는 일단 제외하고 모델 돌려보는 걸 목표로 한다. 

Index(['age', 'ainc', 'HE_sbp', 'HE_dbp', 'HE_ht', 'HE_wt', 'HE_wc', 'HE_glu',
       'HE_chol', 'HE_HDL_st2', 'HE_TG', 'HE_ast', 'HE_alt', 'HE_HB', 'HE_HCT',
       'HE_BUN', 'HE_crea', 'HE_WBC', 'HE_RBC', 'HE_Bplt', 'HE_Uph', 'HE_Usg',
       'sex', 'occp', 'marri_1', 'tins', 'npins', 'D_2_1', 'DI1_pr', 'DI1_pt',
       'DI2_pr', 'DI2_pt', 'DI3_pr', 'DI3_pt', 'DJ4_pr', 'DJ4_pt', 'DE1_pr',
       'DE1_pt', 'BH9_11', 'BH1', 'BH2_61', 'LQ4_00', 'LQ1_sb', 'LQ2_ab',
       'EC_occp', 'EC_stt_1', 'EC_stt_2', 'BO1_1', 'BO2_1', 'BD1', 'BS8_2',
       'BS9_2', 'HE_rPLS', 'HE_Unitr', 'HE_Upro', 'HE_Uglu', 'HE_Uket',
       'HE_Ubil', 'HE_Ubld', 'HE_Uro', 'BM1_1', 'BM1_2', 'BM1_3', 'BM1_4',
       'BM1_5', 'BM1_6', 'BM1_7', 'BM1_8', 'live_t', 'educ', 'BO1', 'HE_obe',
       'incm', 'ho_incm', 'incm5', 'ho_incm5', 'edu', 'cfam', 'house', 'D_1_1',
       'DI3_2', 'BD1_11', 'BD2_1', 'BA2_12', 'BA2_13', 'BP1', 'BS3_1',
       'BE3_31', 'BE5_1', 'BP6_10', 'BP6_31', 'DF2_pr', 'DF2_pt', 'depressed'],


In [143]:
# df[['DF2_pr', 'DF2_pt']].isnull().sum() # 0, 0
df[['BP6_10', 'BP6_31']].isnull().sum() # 35, 75

# 두 변수 모두 1년간 자살 생각 및 자살 시도 여부를 예 아니오로 응답한 질문 
# null 값 채우기에는 위험하기 때문에 일단 null rows drop 
###TODO 현재는 정확률을 일부로 높이기 위해 해당 변수를 X 피쳐에 포함 시킬 것.
"""추후에는 해당 변수를 아예 제거해야 정확한 실험 가능합니다. """
df.drop(df[df['BP6_10'].isna() | df['BP6_31'].isna()].index, inplace=True)
# df.shape

In [144]:
df.groupby('depressed').count() # groupby를 이용해 타겟 y 라벨의 클래스 별 개수를 확인

,age,ainc,HE_sbp,HE_dbp,HE_ht,HE_wt,HE_wc,HE_glu,HE_chol,HE_HDL_st2,...,BA2_12,BA2_13,BP1,BS3_1,BE3_31,BE5_1,BP6_10,BP6_31,DF2_pr,DF2_pt
depressed,,,,,,,,,,,,,,,,,,,,,
0.0,4998,4981,4876,4928,4948,4986,4967,4773,4922,4925,...,4998,4998,4997,4998,4986,4986,4998,4998,4998,4998
1.0,228,228,223,225,219,226,226,208,221,221,...,228,228,228,228,226,228,228,228,228,228


In [145]:
df.isnull().sum().sort_values(ascending = False) # 결측치 개수 확인

EC_occp      490
HE_BUN       304
HE_glu       245
HE_alt       213
HE_Uph       154
            ... 
BO2_1          0
BD1            0
BS8_2          0
BS9_2          0
depressed      0
Length: 94, dtype: int64

In [146]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5226 entries, 0 to 6132
Data columns (total 94 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   age         5226 non-null   float64
 1   ainc        5209 non-null   float64
 2   HE_sbp      5099 non-null   float64
 3   HE_dbp      5153 non-null   float64
 4   HE_ht       5167 non-null   float64
 5   HE_wt       5212 non-null   float64
 6   HE_wc       5193 non-null   float64
 7   HE_glu      4981 non-null   float64
 8   HE_chol     5143 non-null   float64
 9   HE_HDL_st2  5146 non-null   float64
 10  HE_TG       5107 non-null   float64
 11  HE_ast      5144 non-null   float64
 12  HE_alt      5013 non-null   float64
 13  HE_HB       5145 non-null   float64
 14  HE_HCT      5146 non-null   float64
 15  HE_BUN      4922 non-null   float64
 16  HE_crea     5147 non-null   float64
 17  HE_WBC      5145 non-null   float64
 18  HE_RBC      5146 non-null   float64
 19  HE_Bplt     5131 non-null   floa

In [147]:
# Consider Removed columns to create grouped features again
removed = ['HE_PLS_15', 'HE_PLS_30', 'HE_mPLS', 'HE_HbA1c', 'HE_insulin', 'HE_LDL_drct', 'HE_Uacid', 'HE_Ucrea']
num_features = [item for item in num_features if item not in removed]

removed = ['region', 'AC1_yr', 'MH1_yr', 'EC_wht_0', 'LW_ms', 'LW_pr', 'LW_oc', 'BM1_0', 'L_BR_TO', 'L_LN_TO', 'L_DN_TO', 'LK_LB_US', 'LK_LB_EF']
onehot_features = [item for item in onehot_features if item not in removed]

removed = ['OR1']
label_features = [item for item in label_features if item not in removed]

# removed = ["BP_PHQ_1","BP_PHQ_2","BP_PHQ_3","BP_PHQ_4","BP_PHQ_5","BP_PHQ_6","BP_PHQ_7","BP_PHQ_8","BP_PHQ_9","mh_PHQ_S"]
# label_features = [item for item in y_related if item not in removed]

In [148]:
### 결측값 처리 3 - fillna: KNN
from sklearn.impute import KNNImputer
from sklearn.impute import SimpleImputer

X = df.drop('depressed', axis = 1) # DF2_pr 들어가 있는 상태임
X.shape # (79743, 94) 

###TODO 피쳐 별로 결측값 채우는 방법을 달리 할 수 있음 - 추가 EDA 작업
"""
1) num_features: numerical이니 KNN 써도 됨
2) onehot_features, label_features: 보통 1.0 단위로 값이 커짐. --> KNN으로 결측값을 채우면 1.0 단위로 값이 나와야 하는데 1.2 이렇게 채워짐...
-> 다른 방법으로 결측값 채워야 함 -> 최빈값?
"""

# 1) num_features 결측값 처리 - KNN
knn_imp = KNNImputer(n_neighbors=5)
X[num_features] = knn_imp.fit_transform(X[num_features])

# 2) onehot_features, label_features 결측값 처리 - 최빈값(most frequent values)
freq_imp = SimpleImputer(strategy = "most_frequent")
X[onehot_features] = freq_imp.fit_transform(X[onehot_features])
X[label_features] = freq_imp.fit_transform(X[label_features])

In [149]:
X['BA2_12'].value_counts() # 소숫점 단위로 들어간 게 혹시 있는지 확인 -> 없음 good

BA2_12
5.0    2703
8.0    2022
4.0     350
3.0      99
2.0      42
1.0      10
Name: count, dtype: int64

In [150]:
# transform array(X) to dataframe
df_new = pd.DataFrame(X, columns=df.drop('depressed', axis = 1).columns) 
df_new.isna().sum().sort_values(ascending = False)

age       0
HE_Uro    0
live_t    0
BM1_8     0
BM1_7     0
         ..
DI1_pt    0
DI1_pr    0
D_2_1     0
npins     0
DF2_pt    0
Length: 93, dtype: int64

### 피쳐 그룹별 인코딩 작업
1. numeric -> 인코딩 없이 사용, 단 scaling 필요함 (일단 MinMaxScaler 사용)
2. onehot -> onehot 인코딩으로 진행 (categorical)
3. label -> label 인코딩으로 진행 (ordinal)

In [151]:
# 1. numeric # 1. numeric - scaling 
from sklearn.preprocessing import MinMaxScaler 
scaler = MinMaxScaler(feature_range=(0, 1))
df_new[num_features] = scaler.fit_transform(df_new[num_features])

In [152]:
# 2. onehot -> onehot 인코딩으로 진행
df_new = pd.get_dummies(df_new, columns=onehot_features) # sklearn OneHotEncoder 사용도 가능함

In [153]:
df["depressed"].value_counts()

depressed
0.0    4998
1.0     228
Name: count, dtype: int64

In [154]:
# 3. label -> label 인코딩으로 진행 (ordinal)
# label features 중에서 비해당 처리
label_ans_8 = ["DI3_2", "BD1_11", "BD2_11", "BA2_12", "BA2_13", "BP1", "BS3_1","BE3_31", "BE5_1"] # BD2_11 - removed


# 'DI3_2' 변수의 값이 8인 경우, 해당 값을 0으로 바꾸기
df_new['DI3_2'] = df_new['DI3_2'].apply(lambda x: 3 if x == 8 else x)
df_new['BD1_11'] = df_new['BD1_11'].apply(lambda x: 1 if x == 8 else x)
# df_new['BD2_11'] = df_new['BD2_11'].apply(lambda x: 0 if x == 8 else x)
df_new['BA2_12'] = df_new['BA2_12'].apply(lambda x: 5 if x == 8 else x)
df_new['BA2_13'] = df_new['BA2_13'].apply(lambda x: 5 if x == 8 else x)
df_new['BA2_13'] = df_new['BA2_13'].apply(lambda x: 5 if x == 8 else x)
# df_new['BP1'] = df_new['BP1'].apply(lambda x: 5 if x == 8 else x)
df_new['BS3_1'] = df_new['BS3_1'].apply(lambda x: 0 if x == 8 else x)
# df_new['BE3_31'] = df_new['BE3_31'].apply(lambda x: 5 if x == 88 else x)
# df_new['BE5_1'] = df_new['BE5_1'].apply(lambda x: 5 if x == 88 else x)

In [155]:
list(df_new.columns)

['age',
 'ainc',
 'HE_sbp',
 'HE_dbp',
 'HE_ht',
 'HE_wt',
 'HE_wc',
 'HE_glu',
 'HE_chol',
 'HE_HDL_st2',
 'HE_TG',
 'HE_ast',
 'HE_alt',
 'HE_HB',
 'HE_HCT',
 'HE_BUN',
 'HE_crea',
 'HE_WBC',
 'HE_RBC',
 'HE_Bplt',
 'HE_Uph',
 'HE_Usg',
 'incm',
 'ho_incm',
 'incm5',
 'ho_incm5',
 'edu',
 'cfam',
 'house',
 'D_1_1',
 'DI3_2',
 'BD1_11',
 'BD2_1',
 'BA2_12',
 'BA2_13',
 'BP1',
 'BS3_1',
 'BE3_31',
 'BE5_1',
 'BP6_10',
 'BP6_31',
 'DF2_pr',
 'DF2_pt',
 'sex_1.0',
 'sex_2.0',
 'occp_1.0',
 'occp_2.0',
 'occp_3.0',
 'occp_4.0',
 'occp_5.0',
 'occp_6.0',
 'occp_7.0',
 'marri_1_1.0',
 'marri_1_2.0',
 'tins_10.0',
 'tins_20.0',
 'tins_30.0',
 'npins_1.0',
 'npins_2.0',
 'D_2_1_1.0',
 'D_2_1_2.0',
 'DI1_pr_0.0',
 'DI1_pr_1.0',
 'DI1_pr_8.0',
 'DI1_pt_0.0',
 'DI1_pt_1.0',
 'DI1_pt_8.0',
 'DI2_pr_0.0',
 'DI2_pr_1.0',
 'DI2_pr_8.0',
 'DI2_pt_0.0',
 'DI2_pt_1.0',
 'DI2_pt_8.0',
 'DI3_pr_0.0',
 'DI3_pr_1.0',
 'DI3_pr_8.0',
 'DI3_pt_0.0',
 'DI3_pt_1.0',
 'DI3_pt_8.0',
 'DJ4_pr_0.0',
 'DJ4_pr_1.0',

In [156]:
df['DI3_2'].value_counts() # 라벨 인코딩 잘 되었는지 체크

DI3_2
8.0    5112
3.0      61
1.0      32
2.0      21
Name: count, dtype: int64

In [157]:
train_cols = ['age',
 'ainc',
 'HE_sbp',
 'HE_dbp',
 'HE_ht',
 'HE_wt',
 'HE_wc',
 'HE_glu',
 'HE_chol',
 'HE_HDL_st2',
 'HE_TG',
 'HE_ast',
 'HE_alt',
 'HE_HB',
 'HE_HCT',
 'HE_BUN',
 'HE_crea',
 'HE_WBC',
 'HE_RBC',
 'HE_Bplt',
 'HE_Uph',
 'HE_Usg',
 'incm',
 'ho_incm',
 'incm5',
 'ho_incm5',
 'edu',
 'cfam',
 'house',
 'D_1_1',
 'DI3_2',
 'BD1_11',
 'BD2_1',
 'BA2_12',
 'BA2_13',
 'BP1',
 'BS3_1',
 'BE3_31',
 'BE5_1',
 'BP6_10',
 'BP6_31',
 'DF2_pr',
 'DF2_pt',
 'sex_1.0',
 'sex_2.0',
 'occp_1.0',
 'occp_2.0',
 'occp_3.0',
 'occp_4.0',
 'occp_5.0',
 'occp_6.0',
 'occp_7.0',
 'marri_1_1.0',
 'marri_1_2.0',
 'tins_10.0',
 'tins_20.0',
 'tins_30.0',
 'npins_1.0',
 'npins_2.0',
 'D_2_1_1.0',
 'D_2_1_2.0',
 'DI1_pr_0.0',
 'DI1_pr_1.0',
 'DI1_pr_8.0',
 'DI1_pt_0.0',
 'DI1_pt_1.0',
 'DI1_pt_8.0',
 'DI2_pr_0.0',
 'DI2_pr_1.0',
 'DI2_pr_8.0',
 'DI2_pt_0.0',
 'DI2_pt_1.0',
 'DI2_pt_8.0',
 'DI3_pr_0.0',
 'DI3_pr_1.0',
 'DI3_pr_8.0',
 'DI3_pt_0.0',
 'DI3_pt_1.0',
 'DI3_pt_8.0',
 'DJ4_pr_0.0',
 'DJ4_pr_1.0',
 'DJ4_pr_8.0',
 'DJ4_pt_0.0',
 'DJ4_pt_1.0',
 'DJ4_pt_2.0',
 'DJ4_pt_8.0',
 'DE1_pr_0.0',
 'DE1_pr_1.0',
 'DE1_pr_8.0',
 'DE1_pt_0.0',
 'DE1_pt_1.0',
 'DE1_pt_8.0',
 'BH9_11_1.0',
 'BH9_11_2.0',
 'BH1_1.0',
 'BH1_2.0',
 'BH2_61_1.0',
 'BH2_61_2.0',
 'LQ4_00_1.0',
 'LQ4_00_2.0',
 'LQ1_sb_1.0',
 'LQ1_sb_2.0',
 'LQ2_ab_1.0',
 'LQ2_ab_2.0',
 'LQ2_ab_8.0',
 'EC_occp_1.0',
 'EC_occp_2.0',
 'EC_occp_3.0',
 'EC_occp_4.0',
 'EC_occp_5.0',
 'EC_occp_6.0',
 'EC_occp_7.0',
 'EC_occp_8.0',
 'EC_occp_10.0',
 'EC_occp_88.0',
 'EC_stt_1_1.0',
 'EC_stt_1_2.0',
 'EC_stt_1_3.0',
 'EC_stt_1_8.0',
 'EC_stt_2_1.0',
 'EC_stt_2_2.0',
 'EC_stt_2_3.0',
 'EC_stt_2_8.0',
 'BO1_1_1.0',
 'BO1_1_2.0',
 'BO1_1_3.0',
 'BO2_1_1.0',
 'BO2_1_2.0',
 'BO2_1_3.0',
 'BO2_1_4.0',
 'BD1_1.0',
 'BD1_2.0',
 'BS8_2_1.0',
 'BS8_2_2.0',
 'BS8_2_3.0',
 'BS8_2_8.0',
 'BS9_2_1.0',
 'BS9_2_2.0',
 'BS9_2_3.0',
 'BS9_2_8.0',
 'HE_rPLS_1.0',
 'HE_rPLS_2.0',
 'HE_Unitr_0.0',
 'HE_Unitr_1.0',
 'HE_Upro_0.0',
 'HE_Upro_1.0',
 'HE_Upro_2.0',
 'HE_Upro_3.0',
 'HE_Upro_4.0',
 'HE_Upro_5.0',
 'HE_Uglu_0.0',
 'HE_Uglu_1.0',
 'HE_Uglu_2.0',
 'HE_Uglu_3.0',
 'HE_Uglu_4.0',
 'HE_Uglu_5.0',
 'HE_Uket_0.0',
 'HE_Uket_1.0',
 'HE_Uket_2.0',
 'HE_Uket_3.0',
 'HE_Uket_4.0',
 'HE_Ubil_0.0',
 'HE_Ubil_2.0',
 'HE_Ubil_3.0',
 'HE_Ubil_4.0',
 'HE_Ubld_0.0',
 'HE_Ubld_1.0',
 'HE_Ubld_2.0',
 'HE_Ubld_3.0',
 'HE_Ubld_4.0',
 'HE_Ubld_5.0',
 'HE_Uro_0.0',
 'HE_Uro_1.0',
 'HE_Uro_2.0',
 'HE_Uro_3.0',
 'HE_Uro_4.0',
 'BM1_1_0.0',
 'BM1_1_1.0',
 'BM1_1_8.0',
 'BM1_2_0.0',
 'BM1_2_1.0',
 'BM1_2_8.0',
 'BM1_3_0.0',
 'BM1_3_1.0',
 'BM1_3_8.0',
 'BM1_4_0.0',
 'BM1_4_1.0',
 'BM1_4_8.0',
 'BM1_5_0.0',
 'BM1_5_1.0',
 'BM1_5_8.0',
 'BM1_6_0.0',
 'BM1_6_1.0',
 'BM1_6_8.0',
 'BM1_7_0.0',
 'BM1_7_1.0',
 'BM1_7_8.0',
 'BM1_8_0.0',
 'BM1_8_1.0',
 'BM1_8_8.0',
 'live_t_1.0',
 'live_t_2.0',
 'live_t_3.0',
 'live_t_4.0',
 'live_t_5.0',
 'educ_1.0',
 'educ_2.0',
 'educ_3.0',
 'educ_4.0',
 'educ_5.0',
 'educ_6.0',
 'educ_7.0',
 'educ_8.0',
 'BO1_1.0',
 'BO1_2.0',
 'BO1_3.0',
 'BO1_4.0',
 'BO1_5.0',
 'HE_obe_1.0',
 'HE_obe_2.0',
 'HE_obe_3.0',
 'HE_obe_4.0',
 'HE_obe_5.0',
 'HE_obe_6.0']

In [158]:
test_cols = ['age',
 'ainc',
 'HE_sbp',
 'HE_dbp',
 'HE_ht',
 'HE_wt',
 'HE_wc',
 'HE_glu',
 'HE_chol',
 'HE_HDL_st2',
 'HE_TG',
 'HE_ast',
 'HE_alt',
 'HE_HB',
 'HE_HCT',
 'HE_BUN',
 'HE_crea',
 'HE_WBC',
 'HE_RBC',
 'HE_Bplt',
 'HE_Uph',
 'HE_Usg',
 'incm',
 'ho_incm',
 'incm5',
 'ho_incm5',
 'edu',
 'cfam',
 'house',
 'D_1_1',
 'DI3_2',
 'BD1_11',
 'BD2_1',
 'BA2_12',
 'BA2_13',
 'BP1',
 'BS3_1',
 'BE3_31',
 'BE5_1',
 'BP6_10',
 'BP6_31',
 'DF2_pr',
 'DF2_pt',
 'sex_1.0',
 'sex_2.0',
 'occp_1.0',
 'occp_2.0',
 'occp_3.0',
 'occp_4.0',
 'occp_5.0',
 'occp_6.0',
 'occp_7.0',
 'marri_1_1.0',
 'marri_1_2.0',
 'tins_10.0',
 'tins_20.0',
 'tins_30.0',
 'npins_1.0',
 'npins_2.0',
 'D_2_1_1.0',
 'D_2_1_2.0',
 'DI1_pr_0.0',
 'DI1_pr_1.0',
 'DI1_pr_8.0',
 'DI1_pt_0.0',
 'DI1_pt_1.0',
 'DI1_pt_8.0',
 'DI2_pr_0.0',
 'DI2_pr_1.0',
 'DI2_pr_8.0',
 'DI2_pt_0.0',
 'DI2_pt_1.0',
 'DI2_pt_8.0',
 'DI3_pr_0.0',
 'DI3_pr_1.0',
 'DI3_pr_8.0',
 'DI3_pt_0.0',
 'DI3_pt_1.0',
 'DI3_pt_8.0',
 'DJ4_pr_0.0',
 'DJ4_pr_1.0',
 'DJ4_pr_8.0',
 'DJ4_pt_0.0',
 'DJ4_pt_1.0',
 'DJ4_pt_8.0',
 'DE1_pr_0.0',
 'DE1_pr_1.0',
 'DE1_pr_8.0',
 'DE1_pt_0.0',
 'DE1_pt_1.0',
 'DE1_pt_8.0',
 'BH9_11_1.0',
 'BH9_11_2.0',
 'BH1_1.0',
 'BH1_2.0',
 'BH2_61_1.0',
 'BH2_61_2.0',
 'LQ4_00_1.0',
 'LQ4_00_2.0',
 'LQ1_sb_1.0',
 'LQ1_sb_2.0',
 'LQ2_ab_1.0',
 'LQ2_ab_2.0',
 'LQ2_ab_8.0',
 'EC_occp_1.0',
 'EC_occp_2.0',
 'EC_occp_3.0',
 'EC_occp_4.0',
 'EC_occp_5.0',
 'EC_occp_6.0',
 'EC_occp_7.0',
 'EC_occp_8.0',
 'EC_occp_10.0',
 'EC_occp_88.0',
 'EC_stt_1_1.0',
 'EC_stt_1_2.0',
 'EC_stt_1_3.0',
 'EC_stt_1_8.0',
 'EC_stt_2_1.0',
 'EC_stt_2_2.0',
 'EC_stt_2_3.0',
 'EC_stt_2_8.0',
 'BO1_1_1.0',
 'BO1_1_2.0',
 'BO1_1_3.0',
 'BO2_1_1.0',
 'BO2_1_2.0',
 'BO2_1_3.0',
 'BO2_1_4.0',
 'BD1_1.0',
 'BD1_2.0',
 'BS8_2_1.0',
 'BS8_2_2.0',
 'BS8_2_8.0',
 'BS9_2_1.0',
 'BS9_2_2.0',
 'BS9_2_3.0',
 'HE_rPLS_1.0',
 'HE_rPLS_2.0',
 'HE_Unitr_0.0',
 'HE_Unitr_1.0',
 'HE_Upro_0.0',
 'HE_Upro_1.0',
 'HE_Upro_2.0',
 'HE_Upro_3.0',
 'HE_Upro_4.0',
 'HE_Upro_5.0',
 'HE_Uglu_0.0',
 'HE_Uglu_1.0',
 'HE_Uglu_2.0',
 'HE_Uglu_3.0',
 'HE_Uglu_4.0',
 'HE_Uglu_5.0',
 'HE_Uket_0.0',
 'HE_Uket_2.0',
 'HE_Uket_3.0',
 'HE_Uket_4.0',
 'HE_Ubil_0.0',
 'HE_Ubil_4.0',
 'HE_Ubld_0.0',
 'HE_Ubld_1.0',
 'HE_Ubld_2.0',
 'HE_Ubld_3.0',
 'HE_Ubld_4.0',
 'HE_Uro_0.0',
 'HE_Uro_2.0',
 'BM1_1_0.0',
 'BM1_1_1.0',
 'BM1_1_8.0',
 'BM1_2_0.0',
 'BM1_2_1.0',
 'BM1_2_8.0',
 'BM1_3_0.0',
 'BM1_3_1.0',
 'BM1_3_8.0',
 'BM1_4_0.0',
 'BM1_4_1.0',
 'BM1_4_8.0',
 'BM1_5_0.0',
 'BM1_5_1.0',
 'BM1_5_8.0',
 'BM1_6_0.0',
 'BM1_6_1.0',
 'BM1_6_8.0',
 'BM1_7_0.0',
 'BM1_7_1.0',
 'BM1_7_8.0',
 'BM1_8_0.0',
 'BM1_8_1.0',
 'BM1_8_8.0',
 'live_t_1.0',
 'live_t_2.0',
 'live_t_3.0',
 'live_t_4.0',
 'educ_1.0',
 'educ_2.0',
 'educ_3.0',
 'educ_4.0',
 'educ_5.0',
 'educ_6.0',
 'educ_7.0',
 'educ_8.0',
 'BO1_1.0',
 'BO1_2.0',
 'BO1_3.0',
 'BO1_4.0',
 'BO1_5.0',
 'HE_obe_1.0',
 'HE_obe_2.0',
 'HE_obe_3.0',
 'HE_obe_4.0',
 'HE_obe_5.0',
 'HE_obe_6.0']

In [159]:
without_cols = [i for i in train_cols if i not in test_cols]

In [160]:
without_cols

['DJ4_pt_2.0',
 'BS8_2_3.0',
 'BS9_2_8.0',
 'HE_Uket_1.0',
 'HE_Ubil_2.0',
 'HE_Ubil_3.0',
 'HE_Ubld_5.0',
 'HE_Uro_1.0',
 'HE_Uro_3.0',
 'HE_Uro_4.0',
 'live_t_5.0']

In [161]:
df_new.isna().any().any()

False

In [162]:
df_new

,age,ainc,HE_sbp,HE_dbp,HE_ht,HE_wt,HE_wc,HE_glu,HE_chol,HE_HDL_st2,...,BO1_2.0,BO1_3.0,BO1_4.0,BO1_5.0,HE_obe_1.0,HE_obe_2.0,HE_obe_3.0,HE_obe_4.0,HE_obe_5.0,HE_obe_6.0
0,0.327869,0.190829,0.274809,0.438202,0.748281,0.341185,0.321232,0.141935,0.207101,0.304348,...,False,True,False,False,False,False,True,False,False,False
1,0.327869,0.190829,0.122137,0.269663,0.640990,0.150456,0.125413,0.158065,0.316568,0.532609,...,True,False,False,False,True,False,False,False,False,False
3,0.639344,0.662846,0.183206,0.325843,0.407153,0.232523,0.279428,0.222581,0.378698,0.543478,...,False,False,True,False,False,False,False,True,False,False
4,0.606557,0.914588,0.282443,0.314607,0.610729,0.265957,0.323432,0.216129,0.476331,0.445652,...,False,True,False,False,False,False,True,False,False,False
5,0.557377,0.914588,0.167939,0.337079,0.414030,0.108663,0.155116,0.161290,0.562130,0.673913,...,True,False,False,False,False,True,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6128,0.688525,0.196449,0.251908,0.292135,0.411279,0.185410,0.295930,0.187097,0.346154,0.347826,...,False,False,False,True,False,False,True,False,False,False
6129,0.524590,0.325691,0.267176,0.348315,0.266850,0.141337,0.190319,0.177419,0.331361,0.380435,...,False,False,True,False,False,False,True,False,False,False
6130,0.081967,0.325691,0.167939,0.348315,0.320495,0.186170,0.207921,0.187097,0.215976,0.597826,...,False,False,True,False,False,False,False,True,False,False
6131,0.754098,0.121151,0.450382,0.337079,0.635488,0.327508,0.390539,0.254839,0.162722,0.369565,...,False,False,True,False,False,False,False,True,False,False


In [163]:
X = df_new

In [164]:
y = df["depressed"]

In [165]:
y.isnull().sum()

0

In [166]:
y.value_counts()

depressed
0.0    4998
1.0     228
Name: count, dtype: int64

In [109]:
X.to_csv('../../data/HN_X_231206_test.csv', index=False)
y.to_csv('../../data/HN_y_231206_test.csv', index=False)

- 결측치 어떻게 채울지
- test는 비율이 상관없